In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

**References**
- [Switch Transformer](https://arxiv.org/pdf/2101.03961.pdf)
- [Towards Understanding Mixture of Experts in Deep Learning](https://arxiv.org/pdf/2208.02813.pdf)

# Scaling Laws

In our module on [Transformer Scaling](Transformers_Scaling.ipynb) we learned
- that a model with more parameters (denoted $N$)
- is more example-efficient
    - for a given loss level $L$: a large model achieves $L$ with fewer examples than a smaller model
- but at the cost of greater computation $C$
    - each parameters is consumed in some computation

<table>
    <tr>
        <img src="images/scaling_loss_vs_datasize_compute.png">
    </tr>
    
Attribution: https://arxiv.org/pdf/2001.08361.pdf#page=4
</table>

This module describes a technique to facilitate
- large models (greater $N$)
- without increasing computation $C$

# Mixture of Experts (MoE)

Let us consider a binary classification task
- given an example $(\x, \y)$ where
    - $\x$ is from distribution $\mathcal{D}$
    - $\y \in \{ 0, 1\}$
- compute $\pr{ \y = 1 | \x \in \mathcal{D} }$

We would typically solve this task by constructing a Neural Network with $N$ parameters $\Theta$
- computing $p_\Theta( \y = 1 | \x \in \mathcal{D} ) \approx \pr{ \y = 1 | \x \in \mathcal{D} }$

The idea of a *Mixture of Experts (MoE)*
- is to use a union of $M$ distinct neural networks
- where Neural Network with index $m$ is an *expert* in solving the tasks for examples in a subset of $\mathcal{D}$

Neural Network with index $m$
- has its own set of parameters $\Theta_m$, ($0 \le m \le M-1$)
    - each of size $N$
- specializing in classifying a subset $\mathcal{E}_m$ of $\mathcal{D}$

$$
\begin{array} \\
p_{\Theta_m}(\y = 1 | \x \in \mathcal{E}_m) \\
\cup_{m=1}^M \mathcal{E}_m  = \mathcal{D}
\end{array}
$$


It would seem plausible that the Mixture of Experts
- would have lower loss on approximating $\pr{ \y = 1 | \x \in \mathcal{D} }$
- than the single model $p_\Theta$

Unfortunately: this comes at the cost of a factor of $M$ more parameters.

For example, 
Suppose that the $\x \in \mathcal{D}$ naturally form $C$ clusters $\{\mathcal{D}_1, \ldots \mathcal{D}_C \}$
$$
\mathcal{D}  = \cup_{c=1}^C \mathcal{D}_k
$$
- e.g., each cluster is a "topic" (Business, Arts, Science) or language (English, French, Mandarin)

Lettng 
- $M = C$
- $\mathcal{E}_m = \mathcal{D}_m$
- we devote one expert per cluster

In order to make this idea complete, we hypothesize a *router*
- that computes the probability that example $\x$ is best suited to Expert $m$
- computes a vector of length $M$ 
$$
p_0 (\x), \ldots, p_{M-1}(\x) 
$$
- such that 
$$
p_m (\x) = \pr{ \x \in \mathcal{E}_m }
$$

The idea would be to "route" example $\x$ to the expert(s) best suited for its classification.

The Mixture of Experts network could compute the probability weighted sum of the experts
$$
\sum_{m=0}^{M-1} { p_m(\x) * p_{\Theta_m}(\y=1 | \x ) } 
$$


<table>
    <center><strong>Mixture of Experts Layer</strong></center>
    <tr>
        <img src="images/moe_layer.png" width=60%>
    </tr>
    
Attribution: https://arxiv.org/pdf/2208.02813.pdf#page=6
</table>

In an idealized world
- the vector 
would have a single element $m'$ with
$$
p_i (\x) = 
\begin{cases}
 1 & i = m'   \\
 0 & i \ne m' \\
\end{cases}
\text{ where } \x \in \mathcal{E}_{m'} 
$$
- we could somehow avoid computing $p_{\Theta_m}(\y=1 | \x ) $ for experts with $0$ probability  $p_m(\x)$

But, in a non-idealized world where each expert has a non-zero $p_i(\x)$
- the amount of computation is $M$ times that of the single Neural Network
- since each expert $m$ must compute $p_{\Theta_m}(\y=1 | \x ) $

In summary: the naive MoE model increases (by a factor of $M$) both
- parameters
- computation

A way to increase
- number of parameters by factor of $M$
- but increase computation by a *constant* factor $k \ge 1$
- is to sort the probabilities
$$
p_{m_0}(\x) \ge p_{m_1}(\x) \ge \ldots \ge p_{m_{M-1}}(\x)
$$
- and route $\x$ to only the *top-k* experts
    - $T_x = \{ m_0, \ldots, m_{k -1}\}$
- normalizing the $k$ probabilities to sum to $1$

Thus we
- have a bigger model (factor of $M$)
- with constant multiplicative increase in computation

When $k \lt M$, some of the experts are not activated for each example.

We call such a model *sparsely activated* (as opposed to *densely activated*).

# Training an MoE model

The idea sounds simple enough
- construct a sub-network for the router, computing
$$
p_0 (\x), \ldots, p_{M-1}(\x) 
$$
- construct $M$ sub-networks of identical experts (with *separate* weights) 
- jointly train the model on a standard training set of examples $\langle \x, \y \rangle$

We just jointly train the sub-networks to optimize the Loss function.
- [See](https://arxiv.org/pdf/2208.02813.pdf#page=6)


As is the usual practice with Neural Networks
- we are not pre-determining
    - how the router decides which expert is best for a given $\x$
    - which subset $\mathcal{E}_m$ the expert with index $m$ decides to specialize on

It is somewhat of a miracle that joint training works.

For instance, 
the MoE model could collapse to using a single expert $m'$ for all examples
- router ignores $\x$ and computes
    $$
p_i (\x) = 
\begin{cases}
 1 & i = m'   \\
 0 & i \ne m' \\
\end{cases}
\text{ for all } \x 
$$


This would seem to be an issue in the early epochs
- experts have initialization from same distribution
    - each has expected classification probability of $p_{\Theta_m}(\y = 1) = 0.5$
    - minor differences not significant
    - but may influence the router
    

A related issue should be familiar to those familiar with Reinforcement Learning
- trade-off between exploitation and exploration
    - router would favor highest probability expert, even when probability differences are insignificant
        - *exploitation*
    - router might learn of a better routing by only choosing *non-highest-rated expert* with some probability
        - *exploration*

The [hard switching](https://arxiv.org/pdf/2101.03961.pdf#page=8) of the router
can also cause instability
- an insignificant difference in routing probabilities $p_i(\x)$ and $p_{i'}(\x)$ causes $\x$ to 
*always* be routed to expert $i$
- soft choice distributes the probability almost evenly between $i$ and $i'$

In practice, methods to avoid this includes
- [sample from the argmax](https://arxiv.org/pdf/2101.03961.pdf#page=29) (that determines the top-k experts) rather than using deterministic choice
- [add noise](https://arxiv.org/pdf/2208.02813.pdf#page=6) to $p_i(\x)$

Another issue
- the `argmax` (used in choosing top-k)
    - is a hard choice
    - not a soft choice (as in using a sigmoid to implement a [soft](http://localhost:8888/notebooks/NYU/Neural_Programming.ipynb#%22If%22-statements---Gates) if-then-else
    - so is not differentiable
    
I believe this is not an issue
- since the gradient that would be routed to an inactive expert would be $0$ in any event

# Switch Transformer

The Switch Transformer uses a Mixture of Experts
- to replace the Feed Forward Network (FFN) component of a Transformer
- every few Transformer blocks

It chooses $k=1$
- routing to a single expert

<table>
    <center><strong>Switch Transformer Encoder</strong></center>
    <tr>
        <img src="images/switch_transformer_encoder.png" width=75%>
    </tr>
    
Attribution: https://arxiv.org/pdf/2101.03961.pdf#page=5
</table>

## Optimization

The  [Switch Transformer](https://arxiv.org/pdf/2101.03961.pdf) paper is very concerned
with maximizing computational potential.

Suppose the $M$ experts are distributed over several computational units (cores or processors)
- if too many tokens in a mini-batch are routed to the same expert $m$
- expert $m$ becomes a bottle-neck
- while all the computational units of the other experts $m' \ne m$ are idle

Their solution 
- define a *capacity* for each expert
    - number of buffers allocated to each expert: maximum number of tokens it can accommodate
- implement [differentiable load-balancing](https://arxiv.org/pdf/2101.03961.pdf#page=7)
    - add a term to the Loss that penalizes for a non-uniform distribution of tokens to experts
$$
\begin{array} \\
f_m & = & \frac{1}{T} \sum_{\x \in \text{batch}} { 1 * \left( \text{argmax}( p(\x) ) = m \right) } &\text{fraction of tokens dispatached to expert } m \\
P_m & = & \frac{1}{T} \sum_{\x \in \text{batch}} { p_m (\x) } & \text{fraction of probability allocated to expert } m \\
\loss_\text{load balance} & = & \alpha * M * \sum_{m=0}^{M-1} { f_m * P_m } &  \text{minimized when each term is } \frac{1}{M} \\ 
& & & \alpha \text{ is relative importance of load-balancing to other loss terms } \\
& & & \text{multiply by } M \text{ so term is insensitive to number of experts}
\end{array}
$$
- [Note](https://arxiv.org/pdf/2101.03961.pdf#page=8)
    - $P$-vector is differentiable, $f$ vector is not

This is an example of
- practical engineering, supported by math
- rather than theoretical perfection

In [2]:
print("Done")

Done
